In [1]:
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Tải tài nguyên cần thiết cho VADER
nltk.download('vader_lexicon')

# Tải dữ liệu đã được xử lý và gộp từ nguồn UCSD
df = pd.read_csv('ucsd_delaware_reviews_combined.csv')

# Khởi tạo VADER
analyzer = SentimentIntensityAnalyzer()

# Tính toán điểm sentiment (đảm bảo cột 'text' không có giá trị NaN)
df.dropna(subset=['text'], inplace=True)
df['sentiment_score'] = df['text'].apply(lambda text: analyzer.polarity_scores(str(text))['compound'])

print("Tải dữ liệu nâng cấp và tính điểm sentiment thành công!")
print(df[['text', 'sentiment_score', 'category']].head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Tải dữ liệu nâng cấp và tính điểm sentiment thành công!
                                                text  sentiment_score  \
0  Lived here for 3 years and enjoyed it. Locatio...           0.8176   
1  Lived here for 3 years and enjoyed it. Locatio...           0.8176   
2  Nice complex and awesome staff.  Maintenance i...           0.9685   
3  Nice complex and awesome staff.  Maintenance i...           0.9685   
4  Good places for people to live my friend lives...           0.7269   

                category  
0  ['Apartment complex']  
1  ['Apartment complex']  
2  ['Apartment complex']  
3  ['Apartment complex']  
4  ['Apartment complex']  


In [2]:
# --- 1. Đặc trưng Metadata ---
# Độ dài review (số ký tự)
df['review_length'] = df['text'].str.len()

# Đếm số review của mỗi user
# .transform('count') sẽ gán số đếm cho mỗi dòng của user đó
df['user_review_count'] = df.groupby('user_id')['user_id'].transform('count')

# Độ chênh lệch giữa rating của review và rating trung bình của địa điểm
# fillna(0) để xử lý các trường hợp không có avg_rating
df['rating_deviation'] = (df['rating'] - df['avg_rating']).fillna(0)


# --- 2. Suy luận Tín hiệu "Đã ghé thăm" ---
# Các từ khóa cho thấy người dùng đã thực sự đến địa điểm
visit_keywords = [
    'visited', 'went to', 'ate here', 'dined here', 'was there',
    'stayed at', 'my visit', 'our visit', 'ordered', 'tried the'
]
# Tạo cờ (flag) nếu text chứa bất kỳ từ khóa nào trong danh sách trên
df['has_visit_keyword'] = df['text'].str.contains('|'.join(visit_keywords), case=False, na=False)

print("Tạo các đặc trưng mới thành công!")
# Hiển thị các cột mới để kiểm tra
print(df[['user_name', 'review_length', 'user_review_count', 'rating_deviation', 'has_visit_keyword']].head())

Tạo các đặc trưng mới thành công!
        user_name  review_length  user_review_count  rating_deviation  \
0  Heather Carper            112                  4               0.5   
1  Heather Carper            112                  4               0.5   
2  STACY CLAVETTE            283                  5               0.5   
3  STACY CLAVETTE            283                  5               0.5   
4       Zion Hood             74                  5               0.5   

   has_visit_keyword  
0              False  
1              False  
2              False  
3              False  
4              False  


In [3]:
def detect_violations_rules(row):
    flags = {}

    # Chính sách 1: Rant không ghé thăm
    # Điều kiện: sentiment rất tiêu cực VÀ không có từ khóa ghé thăm
    is_rant_no_visit = row['sentiment_score'] < -0.5 and not row['has_visit_keyword']
    flags['is_rant_without_visit'] = is_rant_no_visit

    # Chính sách 2: Nội dung không liên quan (dựa trên quy tắc đơn giản)
    # Điều kiện: Review quá ngắn (có thể là spam hoặc không có giá trị)
    is_irrelevant = row['review_length'] < 20
    flags['is_irrelevant'] = is_irrelevant

    # Thêm cờ 'is_clean' nếu không có vi phạm nào bị phát hiện
    flags['is_clean_by_rules'] = not any([is_rant_no_visit, is_irrelevant])

    return pd.Series(flags)

# Áp dụng hàm cho toàn bộ DataFrame
rule_based_flags = df.apply(detect_violations_rules, axis=1)
df = pd.concat([df, rule_based_flags], axis=1)

print("Đã gắn cờ vi phạm dựa trên luật thành công!")
# Kiểm tra các review bị gắn cờ là "rant không ghé thăm"
print("\nCác review có khả năng là 'Rant không ghé thăm':")
print(df[df['is_rant_without_visit'] == True][['text', 'sentiment_score', 'has_visit_keyword']].head())

Đã gắn cờ vi phạm dựa trên luật thành công!

Các review có khả năng là 'Rant không ghé thăm':
                                                 text  sentiment_score  \
30  I love this place I have severe fibromyalgia m...          -0.5568   
31  I love this place I have severe fibromyalgia m...          -0.5568   
42  I run a law firm. They deposit client trust mo...          -0.8070   
43  I run a law firm. They deposit client trust mo...          -0.8070   
84  Took my Miter saw in to replace the handle bec...          -0.5990   

    has_visit_keyword  
30              False  
31              False  
42              False  
43              False  
84              False  


In [4]:
from google.colab import userdata
import huggingface_hub

try:
    hf_token = userdata.get('HF_TOKEN')
    huggingface_hub.login(token=hf_token)
    print("Đăng nhập Hugging Face thành công!")
except Exception as e:
    print("Lỗi! Hãy chắc chắn bạn đã lưu 'HF_TOKEN' trong Colab Secrets.")

Đăng nhập Hugging Face thành công!


In [5]:
# --- PHƯƠ-NG ÁN B: FALLBACK SỬ DỤNG SCIKIT-LEARN (ĐÃ SỬA LỖI) ---
print("API của LLM không khả dụng, chuyển sang Kế hoạch B: Dùng Scikit-learn.")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split # Import thêm để chia dữ liệu

# 1. Chuẩn bị dữ liệu huấn luyện
def create_label(row):
    if row['is_rant_without_visit']:
        return 'rant_no_visit'
    if row['is_irrelevant']:
        return 'irrelevant'
    return 'clean'

df['rule_based_label'] = df.apply(create_label, axis=1)

# Xử lý các giá trị NaN tiềm ẩn trong cột text_clean ngay từ đầu
df['text_clean'] = df['text_clean'].fillna('')

# Chia dữ liệu thành tập huấn luyện và tập kiểm thử (để đánh giá trong Day 3)
# Chúng ta sẽ huấn luyện mô hình trên 80% dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'],
    df['rule_based_label'],
    test_size=0.2,
    random_state=42,
    stratify=df['rule_based_label'] # Giữ tỷ lệ các nhãn
)

# 2. Xây dựng Pipeline
text_clf_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced')),
])

# 3. Huấn luyện mô hình
print("\nĐang huấn luyện mô hình Logistic Regression...")
text_clf_pipeline.fit(X_train, y_train)
print("Huấn luyện hoàn tất!")

# 4. Đưa ra dự đoán trên toàn bộ dữ liệu (bây giờ sẽ không lỗi)
df['sklearn_classification'] = text_clf_pipeline.predict(df['text_clean'])

# 5. Hiển thị kết quả
print("\nKết quả phân loại bằng Scikit-learn (10 dòng đầu):")
print(df[['text', 'rule_based_label', 'sklearn_classification']].head(10))

# Kiểm tra xem nó có phát hiện đúng các trường hợp "rant" không
print("\nKiểm tra các trường hợp 'rant' do model dự đoán:")
print(df[df['sklearn_classification'] == 'rant_no_visit'][['text', 'sentiment_score']].head())

API của LLM không khả dụng, chuyển sang Kế hoạch B: Dùng Scikit-learn.

Đang huấn luyện mô hình Logistic Regression...
Huấn luyện hoàn tất!

Kết quả phân loại bằng Scikit-learn (10 dòng đầu):
                                                text rule_based_label  \
0  Lived here for 3 years and enjoyed it. Locatio...            clean   
1  Lived here for 3 years and enjoyed it. Locatio...            clean   
2  Nice complex and awesome staff.  Maintenance i...            clean   
3  Nice complex and awesome staff.  Maintenance i...            clean   
4  Good places for people to live my friend lives...            clean   
5  Good places for people to live my friend lives...            clean   
6                                      great layout.       irrelevant   
7                                      great layout.       irrelevant   
8  Positives: Elevator, dog park and maintenance....            clean   
9  Positives: Elevator, dog park and maintenance....            clean   

  sk

In [6]:
# To be done:
# Keyword extraction, topic modeling (LDA)
# timestamp delta
# For ads (regex for URLs/promos)
# Multi-label classification
# Prompt Engineering
# Use Hugging Face Inference

In [7]:
!pip install scikit-learn gensim

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Khởi tạo TF-IDF Vectorizer
# Chúng ta sẽ chỉ xem xét 2000 từ phổ biến nhất để tăng tốc
tfidf_vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')

# Fit trên toàn bộ cột text_clean để học từ vựng và trọng số IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text_clean'].fillna(''))
# Lấy danh sách các từ (features)
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())

def extract_top_keywords(doc, top_n=5):
    """Trích xuất top N từ khóa từ một văn bản dựa trên TF-IDF đã fit."""
    # Transform chỉ văn bản này
    tfidf_vector = tfidf_vectorizer.transform([doc])
    # Sắp xếp các chỉ số của các từ theo điểm TF-IDF
    sorted_indices = np.argsort(tfidf_vector.toarray()).flatten()[::-1]
    # Lấy ra top N từ khóa
    top_keywords = feature_names[sorted_indices[:top_n]]
    return ', '.join(top_keywords)

# Áp dụng hàm này để tạo một cột mới
# Lưu ý: Bước này có thể hơi chậm nếu dữ liệu lớn
df['keywords'] = df['text_clean'].fillna('').apply(extract_top_keywords)

print("Đã trích xuất từ khóa bằng TF-IDF:")
print(df[['text', 'keywords']].head())

Đã trích xuất từ khóa bằng TF-IDF:
                                                text  \
0  Lived here for 3 years and enjoyed it. Locatio...   
1  Lived here for 3 years and enjoyed it. Locatio...   
2  Nice complex and awesome staff.  Maintenance i...   
3  Nice complex and awesome staff.  Maintenance i...   
4  Good places for people to live my friend lives...   

                                           keywords  
0    convenience, views, apartments, lived, enjoyed  
1    convenience, views, apartments, lived, enjoyed  
2  close, complex, apartments, maintenance, located  
3  close, complex, apartments, maintenance, located  
4                 awsome, lives, says, friend, live  


In [9]:
from gensim import corpora
from gensim.models import LdaModel

# Token hóa văn bản (tách thành các từ)
tokenized_data = [text.split() for text in df['text_clean'].fillna('')]

# Tạo từ điển và kho văn bản (corpus)
dictionary = corpora.Dictionary(tokenized_data)
# Lọc các từ quá hiếm hoặc quá phổ biến
dictionary.filter_extremes(no_below=5, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

print("Đã chuẩn bị dữ liệu cho LDA.")

Đã chuẩn bị dữ liệu cho LDA.


In [10]:
# Huấn luyện mô hình LDA để tìm ra 5 chủ đề
# Bước này cũng có thể mất vài phút
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10, random_state=42)

print("Đã huấn luyện xong mô hình LDA. Dưới đây là 5 chủ đề chính:")
for idx, topic in lda_model.print_topics(-1):
    print(f'Topic: {idx} \nWords: {topic}\n')

# Gán chủ đề chính cho mỗi review
def get_dominant_topic(doc):
    bow = dictionary.doc2bow(doc.split())
    topics = lda_model.get_document_topics(bow)
    # Chọn chủ đề có xác suất cao nhất
    dominant_topic = sorted(topics, key=lambda x: x[1], reverse=True)[0][0]
    return dominant_topic

df['topic'] = df['text_clean'].fillna('').apply(get_dominant_topic)

print("\nĐã gán chủ đề cho mỗi review:")
print(df[['text', 'topic']].head())

Đã huấn luyện xong mô hình LDA. Dưới đây là 5 chủ đề chính:
Topic: 0 
Words: 0.048*"the" + 0.035*"to" + 0.024*"you" + 0.023*"i" + 0.023*"they" + 0.023*"and" + 0.023*"a" + 0.019*"is" + 0.018*"have" + 0.014*"of"

Topic: 1 
Words: 0.051*"a" + 0.034*"to" + 0.031*"of" + 0.031*"place" + 0.030*"great" + 0.029*"for" + 0.025*"and" + 0.025*"nice" + 0.016*"good" + 0.013*"very"

Topic: 2 
Words: 0.093*"the" + 0.039*"and" + 0.032*"was" + 0.029*"food" + 0.024*"is" + 0.021*"it" + 0.019*"best" + 0.017*"good" + 0.015*"love" + 0.014*"a"

Topic: 3 
Words: 0.097*"and" + 0.090*"great" + 0.061*"very" + 0.055*"service" + 0.044*"friendly" + 0.043*"staff" + 0.042*"good" + 0.026*"food" + 0.021*"helpful" + 0.021*"always"

Topic: 4 
Words: 0.046*"i" + 0.041*"and" + 0.038*"the" + 0.037*"was" + 0.033*"to" + 0.029*"my" + 0.025*"a" + 0.016*"me" + 0.016*"for" + 0.014*"it"


Đã gán chủ đề cho mỗi review:
                                                text  topic
0  Lived here for 3 years and enjoyed it. Locatio...    

In [11]:
# Chuyển đổi cột 'time' (miligiây) sang định dạng datetime
df['datetime'] = pd.to_datetime(df['time'], unit='ms')

# Sắp xếp DataFrame theo user và thời gian
df = df.sort_values(by=['user_id', 'datetime'])

# Tính toán sự khác biệt về thời gian (tính bằng giây) so với review trước đó của cùng một user
df['time_delta_seconds'] = df.groupby('user_id')['datetime'].diff().dt.total_seconds().fillna(0)

print("Đã tính toán chênh lệch thời gian giữa các review của cùng user:")
print(df[['user_name', 'datetime', 'time_delta_seconds']].head())

Đã tính toán chênh lệch thời gian giữa các review của cùng user:
          user_name                datetime  time_delta_seconds
648     Ronald Keys 2018-03-01 15:56:48.891        0.000000e+00
44104  Don Kelleher 2020-05-21 21:43:37.803        0.000000e+00
14627   Tom Carroll 2020-02-08 01:04:21.369        0.000000e+00
23922   Tom Carroll 2020-10-11 14:24:50.518        2.130243e+07
35287  Jeff Peacock 2017-07-23 02:04:50.769        0.000000e+00


In [12]:
# Tạo một vài review giả có chứa URL để kiểm tra
promo_text_1 = "Great place! visit www.mypromo.com for a 10% discount!"
promo_text_2 = "I loved it, check out my blog at http://myblog.net"
df.loc[len(df)] = df.iloc[0] # Copy một dòng để làm mẫu
df.loc[len(df)-1, 'text'] = promo_text_1
df.loc[len(df)] = df.iloc[1]
df.loc[len(df)-1, 'text'] = promo_text_2

# Định nghĩa lại pattern để bắt URL
url_pattern = r'(https|http|www)[^\s]+'
df['has_url'] = df['text'].str.contains(url_pattern, case=False, na=False)

print(f"Số review có URL sau khi thêm mẫu: {df['has_url'].sum()}")
print("Các review chứa URL:")
print(df[df['has_url']][['text', 'has_url']])

/tmp/ipython-input-4177106586.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['has_url'] = df['text'].str.contains(url_pattern, case=False, na=False)


Số review có URL sau khi thêm mẫu: 7
Các review chứa URL:
                                                    text  has_url
9783   Incredibly delicious!!\nAs always.\nLuv the su...     True
31808  Very disorganized staff unfriendly and unhelpf...     True
58287  I loved it, check out my blog at http://myblog...     True
27067  I live by this place. I don't eat Chinese but ...     True
23844  (Translated by Google) Awwweeeessssooooomeeee ...     True
52771  (Translated by Google) Slowwwww at the pharmac...     True
37895  Time well spent and well worth it.\nEvery wher...     True


In [13]:
# Sửa lại hàm để tạo ra nhiều cột nhãn (multi-label)
def create_multilabels(row):
    labels = []
    # Chính sách 1: Rant không ghé thăm
    if row['sentiment_score'] < -0.5 and not row['has_visit_keyword']:
        labels.append('rant_no_visit')
    # Chính sách 2: Quảng cáo
    if row['has_url']:
        labels.append('ad')
    # Chính sách 3: Không liên quan (VD: chủ đề 4 là chủ đề lạ)
    # Giả sử sau khi xem các chủ đề ở trên, bạn thấy topic 4 là không liên quan
    if row['topic'] == 4: # Thay số 4 bằng chỉ số topic bạn cho là không liên quan
        labels.append('irrelevant')

    # Nếu không có nhãn nào, nó là 'clean'
    if not labels:
        labels.append('clean')

    return labels

df['multilabels'] = df.apply(create_multilabels, axis=1)

print("\nĐã tạo các nhãn đa trị (multi-label):")
print(df[['text', 'multilabels']].tail()) # Xem các review quảng cáo vừa thêm


Đã tạo các nhãn đa trị (multi-label):
                                                    text multilabels
42827  A nice beach!  Fairly busy, arrive early to ge...     [clean]
22297  Good food, better if you eat seafood.  Friendl...     [clean]
22299  Good food, better if you eat seafood.  Friendl...     [clean]
19157                 Food was good, staff was friendly.     [clean]
36516  Always great meats, great prices,  great servi...     [clean]


In [14]:
from huggingface_hub import InferenceClient
import json
import time

# Đảm bảo bạn đã login
# huggingface_hub.login(token=userdata.get('HF_TOKEN'))
client = InferenceClient()

# Prompt đã được cải tiến để xử lý multi-label và yêu cầu output JSON
def classify_review_llm_multilabel(review_text, category):
    prompt = f"""
    As an AI assistant for Google Maps, analyze the following review for a place in the category "{category}".
    A review can have one or more of the following violation labels. If no violations are found, classify it as "clean".

    Possible Labels:
    - "ad": Contains advertisements, promotions, or external links.
    - "irrelevant": The content is not related to the given category.
    - "rant_no_visit": A strong complaint that shows no evidence of a real visit.

    Provide your answer ONLY in a valid JSON format with a single key "labels" which is a list of strings.
    For example: {{"labels": ["clean"]}} or {{"labels": ["ad", "irrelevant"]}}.

    Review Text:
    "{review_text}"

    JSON Output:
    """

    try:
        response = client.text_generation(prompt, model="mistralai/Mistral-7B-Instruct-v0.2", max_new_tokens=100, temperature=0.1)

        json_part = response[response.find('{'):response.rfind('}')+1]
        if json_part:
            return json.loads(json_part)
        else:
            return {"labels": ["error_parsing"]}
    except Exception as e:
        if "is currently loading" in str(e):
            print("Model is loading, retrying...")
            time.sleep(15)
            return classify_review_llm_multilabel(review_text, category)
        return {"labels": [f"error_{str(e)}"]}

# Thử nghiệm trên một mẫu nhỏ, bao gồm cả các review quảng cáo bạn vừa tạo
sample_df_llm = df.tail(10).copy() # Lấy 10 review cuối, bao gồm cả review giả

llm_results = sample_df_llm.apply(
    lambda row: classify_review_llm_multilabel(row['text'], row['category']),
    axis=1
)

sample_df_llm['llm_labels'] = [res.get('labels', ['error']) for res in llm_results]

print("\nKết quả phân loại đa nhãn từ LLM (Mistral):")
print(sample_df_llm[['text', 'multilabels', 'llm_labels']])


Kết quả phân loại đa nhãn từ LLM (Mistral):
                                                    text multilabels  \
25777  First time I tried their food will say it wasn...     [clean]   
39737  I have been going here for years. All the staf...     [clean]   
35036                                            Awesome     [clean]   
1534                              Very helpful and nice.     [clean]   
42774  A nice beach!  Fairly busy, arrive early to ge...     [clean]   
42827  A nice beach!  Fairly busy, arrive early to ge...     [clean]   
22297  Good food, better if you eat seafood.  Friendl...     [clean]   
22299  Good food, better if you eat seafood.  Friendl...     [clean]   
19157                 Food was good, staff was friendly.     [clean]   
36516  Always great meats, great prices,  great servi...     [clean]   

                                              llm_labels  
25777  [error_Model mistralai/Mistral-7B-Instruct-v0....  
39737  [error_Model mistralai/Mistral-7B-Ins

In [15]:
# Lưu DataFrame đã được làm giàu với tất cả các feature và label
final_output_filename = 'final_augmented_reviews_for_day3.csv'
df.to_csv(final_output_filename, index=False)

print(f"Đã lưu thành công DataFrame cuối cùng vào file '{final_output_filename}'.")
print("Đây là đầu vào cần cho Day 3.")

Đã lưu thành công DataFrame cuối cùng vào file 'final_augmented_reviews_for_day3.csv'.
Đây là đầu vào cần cho Day 3.
